Script para envio de mensagens simples  
Premisas:  
1. Carregar bibliotecas necessarias
    1. Pandas
    2. Selenium
    3. Webdriver Manager
    4. Time
2. Carregar arquivo de mensagens en dataframe
3. Limpar dataframe  
    1. Eliminar linhas sem mensagens
    2. Eliinar linhas sem telefones
    3. Eliminar colunas desnecessarias
    4. Obter o total de linhas
    5. Resetar indice
4. Abrir o Navegador
5. Abrir o WhatsappWeb
6. Esperar conexão do celular
7. Iniciar Loop de envio de imagens
    1. Enviar mensagem
    2. Iniciar Loop de envio de imagens

Carregar bibliotecas necessarias

In [1]:
import pandas as pd

# necessary libraries for Chrome operations:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

# modified 29-oct-23
from selenium.webdriver.chrome.service import Service

# pip install webdriver_manager
# This librari updates automatically the Browser Manger (in this case, Chrome)
from webdriver_manager.chrome import ChromeDriverManager

# Necessary to convert messages from ASCII text into URL aceptable addresses (convert special characters, spaces, etc)
import urllib

# Just to get image file name from full path
from pathlib import Path

# Time to allow program wait few seconds during Chrome operations
import time

# To allow randomic waiting times (important to avoid Whatsapp account blocking)
import random

# Datetime to store current date of messages sent
# from datetime import date
import datetime as dt

# importar Tkinter
import tkinter as tk
from tkinter import filedialog as fd

Carregar biblioteca de obejtos clicaveis

In [2]:
import whatsappweb_objects as wwo

In [4]:
# modificações 27-set-2025
# definições de botoes e elementos clicaveis
"""
botao_envio = '[aria-label=\"Enviar\"]'
botao_telefone_errado = '[aria-label=\'O número de telefone compartilhado por url é inválido.\']'
botao_telefone_errado_continuar = '#app > div._aiwn._aiwm.app-wrapper-web.font-fix.os-win.x1h89ln0.xoeyzqq > span:nth-child(3) > div > span > div > div > div > div > div > div.x78zum5.x8hhl5t.x13a6bvl.x13crsa5.x1gabggj.x18d9i69.xaso8d8.xp4054r.xuxw1ft > div > div > div > div.html-div.xdj266r.x14z9mp.xat24cr.x1lziwak.xexx8yu.x18d9i69.x78zum5.xl56j7k.x1ejq31n.x18oe1m7.x1sy0etr.xstzfhl.x1so62im.x1syfmzz.x1ja2u2z.x1s928wv.x1j6awrg.x4eaejv.x1wsn0xg.x1r0yslu.x2q1x1w.xapdjt.xr6f91l.x5rv0tg.x1akc3lz.xikp0eg.x1xl5mkn.x1mfml39.x1l5mzlr.xgmdoj8.x1f1wgk5.x1x3ic1u.xfn3atn.x1pse0pq.x1yxkqql.x9f619.xt8t1vi.x1xc408v.x129tdwq.x15urzxu.x1vqgdyp.x4m7ku4.xyo0t3i.xb0esv5 > div > span'
sinal_de_mais = '[data-icon=\"plus-rounded\"]'
triangulo_envio = '[data-icon=\"wds-ic-send-filled\"]'
fotos_e_videos = '//*[@id="app"]/div[1]/span[6]/div/ul/div/div/div[2]/li/div/input'
"""

'\nbotao_envio = \'[aria-label="Enviar"]\'\nbotao_telefone_errado = \'[aria-label=\'O número de telefone compartilhado por url é inválido.\']\'\nbotao_telefone_errado_continuar = \'#app > div._aiwn._aiwm.app-wrapper-web.font-fix.os-win.x1h89ln0.xoeyzqq > span:nth-child(3) > div > span > div > div > div > div > div > div.x78zum5.x8hhl5t.x13a6bvl.x13crsa5.x1gabggj.x18d9i69.xaso8d8.xp4054r.xuxw1ft > div > div > div > div.html-div.xdj266r.x14z9mp.xat24cr.x1lziwak.xexx8yu.x18d9i69.x78zum5.xl56j7k.x1ejq31n.x18oe1m7.x1sy0etr.xstzfhl.x1so62im.x1syfmzz.x1ja2u2z.x1s928wv.x1j6awrg.x4eaejv.x1wsn0xg.x1r0yslu.x2q1x1w.xapdjt.xr6f91l.x5rv0tg.x1akc3lz.xikp0eg.x1xl5mkn.x1mfml39.x1l5mzlr.xgmdoj8.x1f1wgk5.x1x3ic1u.xfn3atn.x1pse0pq.x1yxkqql.x9f619.xt8t1vi.x1xc408v.x129tdwq.x15urzxu.x1vqgdyp.x4m7ku4.xyo0t3i.xb0esv5 > div > span\'\nsinal_de_mais = \'[data-icon="plus-rounded"]\'\ntriangulo_envio = \'[data-icon="wds-ic-send-filled"]\'\nfotos_e_videos = \'//*[@id="app"]/div[1]/span[6]/div/ul/div/div/div[2]/li/d

Carregar arquivo de mensagens en dataframe

In [5]:
arquivo_de_mensagens = fd.askopenfilename(
    title='Selecione o arquivo Excel com a lista de destinatarios',
    filetypes=[('Arquivo Excel','.xls'),('Arquivo Excel','.xlsx')]
    )

contacts_df = pd.read_excel(arquivo_de_mensagens, sheet_name='CLIENTES')

Carregar lista de imagens

In [6]:
# selecionar arquivos de imagen

images_types = [
        ('Arquivos de imagen','.jpg'),
        ('Arquivos de imagen','.jpeg'),
        ('Arquivos de imagen','.png'),
        ('Arquivos de imagen','.gif'),
        ]
 
imgs_path = sorted(list(fd.askopenfilenames(title='Selecione as imagens a enviar',filetypes=images_types)))

In [7]:
imgs_path

['C:/Users/pablo/OneDrive/Documents/GitHub/SendWhatsAppMessages/Imagens de teste/TESTING IMAGE 1080 X 1080 png.png',
 'C:/Users/pablo/OneDrive/Documents/GitHub/SendWhatsAppMessages/Imagens de teste/TESTING IMAGE 1920 X 1080 png.PNG']

Limpar dataframe

In [8]:
# 1. Eliminar linhas sem mensagens
contacts_df = contacts_df[~contacts_df['MENSAGEM'].isnull()]

# 2. Eliinar linhas sem telefones
contacts_df = contacts_df[~contacts_df['TELEFONE'].isnull()]

# 3. Reset index
contacts_df.reset_index(inplace=True)

# 4. Eliminar colunas desnecessarias
contacts_df = contacts_df[['CLIENTE','TELEFONE','MENSAGEM']]

# 5. Obter o total de linhas
numero_de_mensagens = contacts_df['MENSAGEM'].count()

print('Serão enviadas {} mensagens'.format(numero_de_mensagens))

Serão enviadas 6 mensagens


In [9]:
# visualizar dataframe
contacts_df

,CLIENTE,TELEFONE,MENSAGEM
0,Pablo 1,55(71) 99101-3035,Mensagem de teste para Pablo 1
1,Pablo 2 Wrong Number,54(11)2043430,Telefone errado
2,Pablo 3 Good Number,55(71) 9101-3035,Alem de fórmulas para deixar a mensagem person...
3,Pablo 4 Wrong Number,54(11)2043430,Telefone errado
4,Pablo 5 Mensagem Normal,55(71) 99101-3035,Mensagem de teste para Pablo 5 Mensagem Normal
5,Pablo 7,55(71) 9101-3035,Alem de fórmulas para deixar a mensagem person...


Abrir o Navegador

In [10]:
# Criar uma instancia do Google Chrome
msg_browser = webdriver.Chrome()

Abrir o WhatsappWeb

In [11]:
# Navegar até o WhatsApp Web
msg_browser.get("https://web.whatsapp.com/")
time.sleep(2)

Esperar conexão do celular

In [12]:
# Esperar pela lista de contatos do WhatsApp por X segundos
# Isto indica que podemos começar a enviar mensagens
while len(msg_browser.find_elements(By.ID,"side")) < 1:
    time.sleep(1)

Iniciar Loop de envio de mensagens e imagens

In [ ]:
for j, mensagem in enumerate(contacts_df['MENSAGEM']):
    
    cliente = contacts_df.loc[j,'CLIENTE']
    telefone = contacts_df.loc[j,'TELEFONE']
    print(j, cliente, telefone)

    # Converter a mensagem de ASCII para texto plano para ser usada como URL
    url_mensagem = urllib.parse.quote(f"{mensagem}")

    # Construir o link
    link = f"https://web.whatsapp.com/send?phone={telefone}&text={url_mensagem}"

    # ir no link com a mensagem e o telefone
    msg_browser.get(link)
    time.sleep(3)

    envio_disponivel = False
    telefone_errado = False

    # enquanto não aparecer o botão de envio
    # ou a janela pop_up de telefone errado
    # ficar neste loop
    while (not(envio_disponivel) and not(telefone_errado)):
        
        # capturar se aparece o botão de envio
        try:
            # msg_browser.find_element(By.CSS_SELECTOR,botao_envio)
            msg_browser.find_element(By.CSS_SELECTOR,wwo.objects['send_btn'])
            envio_disponivel = True

        except NoSuchElementException:
            envio_disponivel = False
        
        
        # capturar se aparece telefone errado
        try:
            # msg_browser.find_element(By.CSS_SELECTOR,botao_telefone_errado)
            msg_browser.find_element(By.CSS_SELECTOR,wwo.objects['wrong_number_btn'])
            telefone_errado = True
        except NoSuchElementException:
            telefone_errado = False
        
        
        time.sleep(1)
    
    # uma vez que apareceu um deles
    # tomar ação correspondente
    # estas opções são em teoria mutuamente excludentes
    if envio_disponivel:
        # clicar no botão de envio
        # msg_browser.find_element(By.CSS_SELECTOR,botao_envio).click()
        msg_browser.find_element(By.CSS_SELECTOR,wwo.objects['send_btn']).click()
        time.sleep(2)

        # agora começar a enviar as imagens
        for i, img_file in enumerate(imgs_path):
            # buscar sinal de + (antigo clip)
            
            # msg_browser.find_element(By.CSS_SELECTOR,sinal_de_mais).click() 
            msg_browser.find_element(By.CSS_SELECTOR,wwo.objects['add_attachment_btn']).click()
            time.sleep(1)

            # abrir a janela para selecionar o arquivo e enviar as teclas
            # msg_browser.find_element(By.XPATH,fotos_e_videos).send_keys(img_file)
            msg_browser.find_element(By.XPATH,wwo.objects['pictures_and_videos_btn']).send_keys(img_file)
            time.sleep(1)
            # clicar no triangulo de enviar (es mejor que el Enter)
            # msg_browser.find_element(By.CSS_SELECTOR,triangulo_envio).click()
            msg_browser.find_element(By.CSS_SELECTOR,wwo.objects['send_attachment_btn']).click()
            time.sleep(1)

    if telefone_errado:
        # clicar no botao de ok para continuar
        # msg_browser.find_element(By.CSS_SELECTOR,botao_telefone_errado_continuar).click()
        msg_browser.find_element(By.CSS_SELECTOR,wwo.objects['wrong_number_continue_btn']).click()
        time.sleep(2)

    # aguardar um pouco ate a proxima mensagem
    time.sleep(random.randint(3,7))  

0 Pablo 1 55(71) 99101-3035
1 Pablo 2 Wrong Number 54(11)2043430
2 Pablo 3 Good Number 55(71) 9101-3035
3 Pablo 4 Wrong Number 54(11)2043430
4 Pablo 5 Mensagem Normal 55(71) 99101-3035
5 Pablo 7 55(71) 9101-3035


Debug passo a passo

In [ ]:
# pegar a linha j
j = 2
mensagem = contacts_df.loc[j,'MENSAGEM']
cliente = contacts_df.loc[j,'CLIENTE']
telefone = contacts_df.loc[j,'TELEFONE']
print(j, mensagem, cliente, telefone)

In [ ]:
# Converter a mensagem de ASCII para texto plano para ser usada como URL
url_mensagem = urllib.parse.quote(f"{mensagem}")

# Construir o link
link = f"https://web.whatsapp.com/send?phone={telefone}&text={url_mensagem}"

In [ ]:
# ir no link com a mensagem e o telefone e esperar
msg_browser.get(link)

# TAL VEZ ESTA E A SOLUÇÃO
# 06-OCT-2024
# Esperar pela lista de contatos do WhatsApp por X segundos
# Isto indica que podemos começar a enviar mensagens
while len(msg_browser.find_elements(By.ID,"side")) < 1:
    time.sleep(1)

# COLOCAR ISTO NUMA FUNÇÃO PARA REUTILIZAR

In [ ]:

# verificar se pode ser enviado ou se for um numero errado
telefone_errado = False
envio_disponivel = False

while (not(telefone_errado) and not(envio_disponivel)):
    # capturar se aparece o botão de envio
    try:
        time.sleep(1)
        msg_browser.find_element(By.CSS_SELECTOR,botao_envio)
        envio_disponivel = True
        print("pegou envio disponivel")
        break   # SE JA SE QUE ENCONTREI O BOTAO DE ENVIO, NAO FAZ SENTIDO ESPERAR PELO TELEFONE ERRADO
    except NoSuchElementException:
        envio_disponivel = False
        print("nao encontrou enviar")
    
    # capturar telefone errado
    try:
        time.sleep(1)
        msg_browser.find_element(By.CSS_SELECTOR,botao_telefone_errado)
        telefone_errado = True
        print("pegou telefone errado")
        break   # SE JA SEI QUE O TELEFONE ESTA ERRADO, NAO FAZ SENTIDO ESPERAR PELO ENVIO
    except NoSuchElementException:
        telefone_errado = False
        print("nao e telefone errado")
    


print("Telefone errado: ", telefone_errado)
print("Envio disponivel: ", envio_disponivel)

In [ ]:
# modificações 27-set-2025
# se aparece botao de envio, clicar nele
if envio_disponivel:
    msg_browser.find_element(By.CSS_SELECTOR,botao_envio).click()
# se aparece telefone errado, clicar nele
if telefone_errado:
    msg_browser.find_element(By.CSS_SELECTOR,botao_telefone_errado_continuar).click()
time.sleep(2)

In [ ]:
img_file = imgs_path[1]

In [ ]:
img_file

In [ ]:
# click on add attachment (plus sign)
msg_browser.find_element(By.CSS_SELECTOR,sinal_de_mais).click() 

In [ ]:
# clicar em "fotos e videos"
# o xpath tem que ter a palavra "input"
msg_browser.find_element(By.XPATH,fotos_e_videos).send_keys(img_file)

In [ ]:
# clicar no triangulo de envio
msg_browser.find_element(By.CSS_SELECTOR,triangulo_envio).click()

In [ ]:
texto = "Não recebeu a mensagem"

print(texto[:3] == "Não")

In [ ]:
try:
    # Code that might raise an exception
    x = 1 / 0  # This will raise a ZeroDivisionError
except Exception as e:
    # Print the error
    print(f"An error occurred: {e}")


In [ ]:
import traceback
import sys

try:
    x = 1 / 0  # This will raise an exception
except Exception as e:
    exc_type, exc_value, exc_traceback = sys.exc_info()
    line_number = exc_traceback.tb_lineno  # Get the line number within the cell
    print(f"Error: {e}, occurred in cell at line {line_number}")
    traceback.print_exc()  # Print the full traceback


In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
wait = WebDriverWait(msg_browser,20)
# element = WebDriverWait(msg_browser, 20).until(EC.presence_of_element_located((By.ID, "submit"))

In [ ]:
mais = "/html/body/div[1]/div/div/div[3]/div[4]/div/footer/div[1]/div/span/div/div[1]/div[2]/div/div/div/span"

element = WebDriverWait(msg_browser, 200).until(EC.presence_of_element_located((By.XPATH, mais)))

element.click()
